In [2]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm.session import Session, sessionmaker
from sqlalchemy import create_engine
from sqlalchemy.schema import MetaData, Table, Column, ForeignKey
from sqlalchemy.types import Integer, Text
from sqlalchemy.orm import relationship

In [1]:
import pandas as pd

medicine = pd.read_csv('C:/Users/KimYM/Documents/카카오톡 받은 파일/merged_med.csv')
medicine.fillna('-', inplace=True)
medicine.replace('-', None,inplace=True)

In [ ]:
base.metadata.clear(), engine.dispose()

In [3]:
engine = create_engine('sqlite:///C:/Users/KimYM/Documents/Medicine/medicine_db.db')
meta = MetaData(engine)
base = declarative_base()

In [ ]:
engine = create_engine('sqlite:///:memory:')
meta = MetaData(engine)
base = declarative_base()

In [4]:
class Medicine(base):
    __tablename__ = 'T_Medicine'
    
    pk = Column('pk', Integer, primary_key=True)
    name = Column('name', Text)
    content_url = Column('content_url', Text)
    identifier_front = Column('identifier_front', Text)
    identifier_back = Column('identifier_back', Text)
    color_front = Column('color_front', Text)
    color_back = Column('color_back', Text)
    shape = Column('shape', Text)
    form_code = Column('form_code', Text)
    image_url = Column('image_url', Text)
    company_name = Column('company_name', ForeignKey('T_Company.name'), nullable=False)
    
    company = relationship('Company', back_populates='medicines')
    logos = relationship('Logo_Medi', back_populates='medicine', uselist=True)
    
    def __init__(self, name, company, content_url=None, iden_front=None, iden_back=None, form_code=None,
                 image_url=None, color_front=None, color_back=None, shape=None,logos=[]):
        self.name = name
        self.content_url = content_url
        self.identifier_front = iden_front
        self.identifier_back = iden_back
        self.color_front = color_front
        self.color_back = color_back
        self.shape = shape
        self.form_code = form_code
        self.image_url = image_url
        self.company_name = company

        session.add(self)
        session.commit()
        
        if len(logos):
            for logo_num in logos:
                if session.query(Logo.logo_num).filter(Logo.logo_num == logo_num).first() is not None:
                    new_link = Logo_Medi(Lfk=logo_num, Mfk=self.pk)
                    session.add(new_link)
                    session.commit()
        session.add(self)
        session.commit()
        
        if session.query(Company.name).filter(Company.name == company).first() is None:
            Company(company)
        
    def info(self):
        print('제조사:', self.company_name)
        print('품목명:', self.name)
        print('식별문자앞:', self.identifier_front)
        print('식별문자뒤:', self.identifier_back)
        print('색상앞:', self.color_front)
        print('색상뒤:', self.color_back)
        print('모양:', self.shape)
        print('제형코드이름:', self.form_code)
        for i, _ in enumerate(self.logos):
            print(f'로고번호{i}:', _.Lfk)
        print('상세페이지:', self.content_url)
        print()
        
    
class Company(base):
    __tablename__ = 'T_Company'
    
    name = Column('name', Text, primary_key=True, autoincrement=False)
    
    medicines = relationship('Medicine', back_populates='company', uselist=True)
    
    def __init__(self, name):
        self.name = name
        session.add(self)
        session.commit()

    def get_medicines(self):
        medi = list()
        print('제조사:', self.name)
        for medicine in self.medicines:
            print('    제품명:', medicine.name)
        print()
            
class Logo(base):
    __tablename__ = 'T_Logo'
    
    logo_num = Column('logo_num', Text, primary_key=True, autoincrement=False)
    image_path = Column('image_path', Text, nullable=False)
    
    medicines = relationship('Logo_Medi', back_populates='logo', uselist=True)
    
    def __init__(self, logo_num, path):
        self.logo_num = logo_num
        self.image_path = path
        session.add(self)
        session.commit()
        
    def get_medicines(self):
        medi = list()
        print('로고번호:', self.logo_num)
        print('이 로고를 사용하는 제품들')
        for medicine in self.medicines:
            m = session.query(Medicine).filter(Medicine.pk == medicine.Mfk).first()
            print('    제품명:', medicine.pk)
        print('이미지:', self.image_path)
        print()
        
class Logo_Medi(base):
    __tablename__ = 'T_Logo_Medi'
    
    pk = Column('pk', Integer, primary_key=True)
    Lfk = Column('Lfk', Text, ForeignKey('T_Logo.logo_num'), nullable=False)
    Mfk = Column('Mfk', Integer, ForeignKey('T_Medicine.pk'), nullable=False)
    
    logo = relationship('Logo', back_populates='medicines')
    medicine = relationship('Medicine', back_populates='logos')

In [5]:
Session = sessionmaker(engine)
session = Session()
base.metadata.create_all(engine)

In [6]:
import os

In [7]:
path = './logo_num_image/'
file_list = os.listdir(path)

for file in file_list:
    logo_num = file[:5]
    Logo(logo_num, path+file)

In [8]:
for i, pill_name in enumerate(medicine.품목명):
    logos = list()
    if medicine.표기코드앞[i] != None:
        logos.append(medicine.표기코드앞[i])
        
    if medicine.표기코드뒤[i] != None:
        logos.append(medicine.표기코드뒤[i])
        
    try:
        Medicine(pill_name, medicine.업소명[i], content_url=medicine.url[i],
             iden_front=medicine.표시앞[i], iden_back=medicine.표시뒤[i], form_code=medicine.제형코드명[i],
             image_url='./medicine_image/'+str(medicine.품목일련번호[i])+'.jpg',
             color_front=medicine.색상앞[i], color_back=medicine.색상뒤[i],
             shape=medicine.의약품제형[i],logos=logos)
    except:
        print(pill_name)
        raise

In [ ]:
session.query(Logo.logo_num).filter(Logo.logo_num=='').first() is None

In [ ]:
len(session.query(Medicine).all())

In [ ]:
for c in session.query(Company).all()[:30]:
    c.get_medicines()


In [ ]:
file_list[0][:5]

In [ ]:
Medicine('name1', 'company2', logos=['r0001'])

In [ ]:
for logo in session.query(Medicine).first().logos:
    print(logo.Lfk)

In [ ]:
session.query(Medicine).first().info()

In [ ]:
session.query(Company).first().get_medicines()

In [ ]:
Medicine('name1', 'company2', logo_num='r0001')
Medicine('name2', 'company1', logo_num=4)
Medicine('name3', 'company3', logo_num=1)
Medicine('name4', 'company4', logo_num=3)
Medicine('name5', 'company1', logo_num=2)
Medicine('name6', 'company3', logo_num=1)
Medicine('name7', 'company2', logo_num=2)
Medicine('name8', 'company1')

In [ ]:
for _ in session.query(Company).all():
    _.get_medicines()

In [ ]:
for _ in session.query(Logo).all():
    _.get_medicines()

In [ ]:
medicine[medicine.표시뒤.isna()]

In [ ]:
medicine.표시뒤.value_counts()

In [ ]:
def __init__(self, name, company, content_url=None, iden_front=None, iden_back=None, form_code=None,
                 image_url=None, color=None, shape=None,logo_num=None):

In [ ]:
medicine.info()

In [ ]:
medicine.표기코드앞[245]

In [ ]:
for pill in medicine.품목명:
    print(pill)